In [1]:
import sys
sys.path.append("../../XLEMOO")

In [7]:
from XLEMOO.LEMOO import EAParams, MLParams, LEMParams, LEMOO, PastGeneration
from XLEMOO.fitness_indicators import asf_wrapper
from XLEMOO.selection import SelectNBest
from XLEMOO.plotting import show_rules
from XLEMOO.ruleset_interpreter import extract_skoped_rules
from desdeo_emo.recombination import SBX_xover, BP_mutation
from desdeo_tools.scalarization.ASF import PointMethodASF
from desdeo_problem.testproblems import vehicle_crashworthiness

import matplotlib.pyplot as plt
import numpy as np
from imodels import SkopeRulesClassifier

## Initialize the problem

In [3]:
n_objectives = 3
n_variables = 5

problem = vehicle_crashworthiness()

## Initialize an XLEMOO method

In [112]:
ideal = np.array([1600.0, 6.0, 0.038])
nadir = np.array([1700.0, 12.0, 0.2])
# ref_point = np.array([1650.0, 7.0, 0.05])  # the reference point
ref_point = np.array([1690.0, 6.5, 0.039])  # the reference point

# define the achievement scalarizing function as the fitness function
ref_asf = asf_wrapper(PointMethodASF(ideal=ideal, nadir=nadir), {"reference_point": ref_point})
fitness_fun = ref_asf

lem_params = LEMParams(
    use_darwin=True,
    use_ml=True,
    fitness_indicator=fitness_fun,
    ml_probe = 1,
    ml_threshold = None,
    darwin_probe = None,
    darwin_threshold = None,
    total_iterations=15,
)

ea_params = EAParams(
    population_size=50,
    cross_over_op=SBX_xover(),
    mutation_op=BP_mutation(problem.get_variable_lower_bounds(), problem.get_variable_upper_bounds()),
    selection_op=SelectNBest(None, 50),  # keep population size constant
    population_init_design="LHSDesign",
    iterations_per_cycle=19,
)

ml = SkopeRulesClassifier(precision_min=0.1, n_estimators=30, max_features=None, max_depth=None, bootstrap=True, bootstrap_features=True)
ml_params = MLParams(
    H_split=0.10,
    L_split=0.10,
    ml_model=ml,
    instantation_factor=10,
    generation_lookback=0,
    ancestral_recall=0,
    unique_only=True,
    iterations_per_cycle=1,
)

lemoo = LEMOO(problem, lem_params, ea_params, ml_params)

## Run the XLEMOO method

In [113]:
lemoo.run_iterations()

h_gorup shape: (48, 5)
l_gorup shape: (48, 5)
.
h_gorup shape: (83, 5)
l_gorup shape: (83, 5)
.
h_gorup shape: (113, 5)
l_gorup shape: (113, 5)
.
h_gorup shape: (140, 5)
l_gorup shape: (140, 5)
.
h_gorup shape: (166, 5)
l_gorup shape: (166, 5)
.
h_gorup shape: (196, 5)
l_gorup shape: (196, 5)
.
h_gorup shape: (226, 5)
l_gorup shape: (226, 5)
.
h_gorup shape: (256, 5)
l_gorup shape: (256, 5)
.
h_gorup shape: (287, 5)
l_gorup shape: (287, 5)
.
h_gorup shape: (318, 5)
l_gorup shape: (318, 5)
.
h_gorup shape: (348, 5)
l_gorup shape: (348, 5)
.
h_gorup shape: (375, 5)
l_gorup shape: (375, 5)
.
h_gorup shape: (402, 5)
l_gorup shape: (402, 5)
.
h_gorup shape: (427, 5)
l_gorup shape: (427, 5)
.
h_gorup shape: (446, 5)
l_gorup shape: (446, 5)
.


{'darwin_mode': 285, 'learning_mode': 15, 'total_iterations': 15}

## Extract most accurate rules

In [114]:
rules, accuracies = extract_skoped_rules(lemoo.current_ml_model)
problem_lower_bounds = problem.get_variable_lower_bounds()
problem_upper_bounds = problem.get_variable_upper_bounds()

rules_for_vars = {f"X_{i}": {">": [problem_lower_bounds[i], -1], "<=": [problem_upper_bounds[i], -1]} for i in range(n_variables)}

for accuracy, rule in zip(accuravies, rules):
    for key in rule:
        var_name = key[0]
        op = key[1]
        
        # check accuracy
        if rules_for_vars[var_name][op][1] < accuracy: 
            # update accuracy
            rules_for_vars[var_name][op][1] = accuracy
            if op == "<=":
                # tighten rule, if necessary
                if float(rule[(var_name, op)]) <= rules_for_vars[var_name][op][0]:
                    rules_for_vars[var_name][op][0] = float(rule[(var_name, op)])
            elif op == ">":
                # tighten rule, if necessary
                if float(rule[(var_name, op)]) > rules_for_vars[var_name][op][0]:
                    rules_for_vars[var_name][op][0] = float(rule[(var_name, op)])
            
for rule in rules_for_vars:
    print(f"{rule}: {rules_for_vars[rule]}")

X_0: {'>': [1.00219, 0.49947379311924145], '<=': [1.00893, 0.9475806451612904]}
X_1: {'>': [2.99815, 1.0], '<=': [3.0, -1]}
X_2: {'>': [1.0, -1], '<=': [1.00022, 1.0]}
X_3: {'>': [1.0, -1], '<=': [1.25205, 1.0]}
X_4: {'>': [2.99784, 1.0], '<=': [3.0, -1]}


## Extract bounds of the final population

In [110]:
lower_bounds = np.min(lemoo._generation_history[-1].individuals, axis=0)
upper_bounds = np.max(lemoo._generation_history[-1].individuals, axis=0)

bounds_for_vars = {f"X_{i}": {">": lower_bounds[i], "<=": upper_bounds[i]} for i in range(n_variables)}

bounds_for_vars

{'X_0': {'>': 1.0018813959737811, '<=': 1.0018814114835006},
 'X_1': {'>': 2.9999999999988414, '<=': 3.0},
 'X_2': {'>': 1.0000000000004012, '<=': 1.0000000001099774},
 'X_3': {'>': 1.2487838075325863, '<=': 1.2487838100073767},
 'X_4': {'>': 2.999999999893158, '<=': 3.0}}

In [111]:
lemoo._generation_history[-1].objectives_fitnesses[0]

array([1.67718912e+03, 7.60398728e+00, 6.88088390e-02])